In [1]:
import pandas as pd
import os
import psycopg2
import numpy as np
import dask.dataframe as dd
# import sqlalchemy as sql

In [2]:
# Create Connection to postgresql

conn = psycopg2.connect(
    host="localhost",
    database="musicbrainz_db",
    user="musicbrainz",
    port=25432)

cursor = conn.cursor()

# psql -h localhost -p 25432 -U musicbrainz musicbrainz_db

# engine = sql.create_engine('postgresql:///musicbrainz_db')
# connection = engine.connect()

In [6]:
#Load up MLHD Data Folder
filelist = os.listdir('/data/mlhd/00')

path = lambda name: '/data/mlhd/00/{}'.format(name)
file_paths = [path(name) for name in filelist]

first_10 = file_paths[:10]

In [3]:
# fetch df

# Use dask.dataframe since it's ~ 14x faster than pandas dataframe! 

def fetch_df_dd(pth):
    df = dd.read_csv(pth, sep='\t', names=['timestamp', 'artist-MBID', 'release-MBID', 'recording-MBID'])
    df = df[-df['recording-MBID'].isna()]
    return df

df = fetch_df_dd(file_paths[3])
df.head()

NameError: name 'file_paths' is not defined

In [18]:
# A function to take single mbid as input and: 
# 1. cross check with recording table. If present, return the SAME recording-mbid from recording table.
# 2. If not in recording table, check in track table. If present, return 
# 3. If not in recording/track table, add it the list "bad_entries"

track_mbids = []
bad_entries = []
def check_recording(rec_mbid):
    # Check if MBID exists in recording table.
    cursor.execute("SELECT gid FROM recording WHERE gid=%(id)s;", {'id':rec_mbid})
    fetch = cursor.fetchone()
    try:
        # If returned object is subscriptable (i.e. not null), return it.
        return fetch[0]
    except:
        # Else, check if its a track-mbid
        cursor.execute("SELECT gid FROM track WHERE gid=%(id)s;", {'id':rec_mbid})
        fetch = cursor.fetchone()
        try:
            # if subscriptable, it's indeed a track ID. store track-ID somewhere and Return None.
            fetch[:]
            track_mbids.append(fetch[0])
            return None
        
        except:
                # Check if MBID exists in recording_gid_redirect. If yes, return original gid.
                # Else, return None and 
                cursor.execute('''select r.gid from recording r join recording_gid_redirect rgr 
                on rgr.new_id = r.id 
                where rgr.gid = %(id)s;''', {'id':rec_mbid})
                
                fetch = cursor.fetchone()
                
                try:
                    fetch[:]
                    return fetch[0]
            
                except:
                    return None
                
            bad_entries.append(rec_mbid)

# Simple function to take single mbid as input and:
# Check 
def get_canon(rec_mbid):
    cursor.execute("SELECT canonical_recording_mbid FROM mapping.canonical_recording WHERE recording_mbid=%(id)s;", {'id':rec_mbid})
    if cursor.fetchone()!=None:
        return cursor.fetchone()
    else:
        return rec_mbid

In [7]:
x = df['recording-MBID'].map(check_recording)
# check_recording(df['recording-MBID'][1])

In [8]:
# cursor.execute("SELECT canonical_recording_mbid FROM mapping.canonical_recording WHERE recording_mbid=%(id)s;", {'id':'28d24dab-cc0c-45fe-bbc7-cd7e01da03a9'})
# cursor.fetchone()
bad_entries

['c0aec352-a3fa-4a6f-a808-73b7dd7cf6b4',
 '1856a502-b4fa-494e-b13c-1d6868736516',
 '07c46fe4-9e0d-4396-827f-517e95e6dbf4',
 '9e25b892-8431-441d-8318-627d314e5326',
 '822818e1-07fd-4cb3-9dd6-4bfb7f82d7f0',
 '2a9ceceb-2f43-480f-b6da-2c772f8502aa',
 '65f8bcce-e67b-4596-b3c6-f0acd799899e',
 'fee882dd-040b-44b0-8f3d-3cc83586ee7f',
 '0b7ae393-3dc1-47e1-99a9-a8b10f6f867e',
 '02d060e4-c2dd-4ab7-9493-28920d602f3a',
 'fee882dd-040b-44b0-8f3d-3cc83586ee7f',
 '6af7a16e-ed42-4040-a4f5-36ac770ee157',
 '075bc4e5-d06c-4e49-8e12-5a4e790250f2',
 '3f504d26-660b-4d79-a523-2ff25d08f653',
 '3f55af07-78d0-403a-b7b1-d54f202ae7eb',
 '1c994058-3718-47be-831c-3f81ad509ce9',
 '0689f100-1a49-4474-8da1-b42bdc95df17',
 '3a6785e2-20ad-4a05-ac70-2b8f22a02b23',
 '58805e01-cbae-42de-b9ad-c0dd77cfe569',
 '73585c46-72c3-4484-8fcf-2ffb837587d6',
 '566fc8e8-2b34-43de-8332-94d523a9df1c',
 '0b2bb849-00b0-4333-abc2-5518e3bc923a',
 '15564334-0228-4a51-b8c2-f079f661ff73',
 '3f504d26-660b-4d79-a523-2ff25d08f653',
 '19580f75-0592-

In [11]:
with open ('bad_recording_MBIDs.txt', 'w+') as f:
    for item in bad_entries:
        f.write(item+"\n")